<a href="https://colab.research.google.com/github/hnidey13/tesis_SVD_t-SVDM/blob/main/marco_star_M.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Limpiamos área de trabajo

In [157]:
rm(list=ls())

Para mostrar ejemplos, definimos:


*   Tensor $\boldsymbol{\mathcal{A}}\in\mathbb{R}^{4\times5\times3\times6\times2}$
*   Tensor $\boldsymbol{\mathcal{B}}\in\mathbb{R}^{5\times2\times3\times6\times2}$
*   Matrices de transformación $\boldsymbol{M}_1,\boldsymbol{M}_2,\boldsymbol{M}_3$



In [158]:
set.seed(123)
n <- c(4,5,3,6,2)
n2 <- c(5,2,3,6,2)
A <- array(runif(prod(n)),n)
B <- array(rnorm(prod(n2)),n2)
dim_A <- dim(A)
len_dim_A <- length(dim_A)
M_l <- lapply(3:len_dim_A, function(i) matrix(runif(n[i]*n[i]),n[i],n[i]))

**Funciones:** desdoble y doblez

In [159]:
unfold <- function(A,d) t(matrix(unlist(apply(A, d, c, simplify = FALSE)),prod(dim(A)[-d])))
fold <- function(A_d, d, dim_A){
    pos <- append(2:length(dim_A), 1, d-1)
    aperm(array(A_d, dim_A[order(pos)]),pos)
}

**Ejemplo:** desdoble de forma 2

In [160]:
str(unfold(A,2))

 num [1:5, 1:144] 0.288 0.94 0.551 0.678 0.246 ...


**Ejemplo:** doblez de forma 2, para obtener el tensor original

In [161]:
fold2_A2 <- fold(unfold(A,2),2,dim_A)
str(fold2_A2)
paste("Error numérico", sum(abs(fold2_A2-A)))

 num [1:4, 1:5, 1:3, 1:6, 1:2] 0.288 0.788 0.409 0.883 0.94 ...


[1] "Error numérico 0"

**Función:** producto en términos de cara

In [162]:
f_face_prod <- function(A,B){
    #Verificamos dimensión de los tensores
    if (!identical(dim(A)[-1],dim(B)[-2])){
        stop("Verificar dimensiones")
    } else{
        
        A_faces <- apply(A, 3:length(dim(A)), c, simplify = FALSE)
        A_faces <- lapply(A_faces, matrix, dim(A)[1:2])
        
        B_faces <- apply(B, 3:length(dim(B)), c, simplify = FALSE)
        B_faces <- lapply(B_faces, matrix, dim(B)[1:2])
        
        face_prod <- sapply(1:length(A_faces), function(i) A_faces[[i]] %*% B_faces[[i]],
                            simplify = FALSE)
        
        face_prod <- array(unlist(face_prod), c(dim(face_prod[[1]]), dim(A)[-(1:2)]))
        face_prod
    }
}

**Ejemplo:** producto en términos de cara

In [163]:
str(f_face_prod(A,B))

 num [1:4, 1:2, 1:3, 1:6, 1:2] 1.6 0.614 0.895 2.226 0.241 ...


**Función:** producto de forma $k$

In [164]:
prod_k <- function(A, M, k) fold(M %*% unfold(A,k),k,dim(A))

**Ejemplo:** producto de forma $3$

In [165]:
str(prod_k(A, M_l[[1]], 3))

 num [1:4, 1:5, 1:3, 1:6, 1:2] 0.495 0.489 0.406 0.65 0.476 ...


**Función:** traslado al domino transformado

In [166]:
hat_ten <- function(A,M_l){
    len_dim_A <- length(dim(A))
    A_hat <- A
    for (i in 3:len_dim_A){
        A_hat <- prod_k(A_hat, M_l[[i-2]],i)
    }
    A_hat
}

**Ejemplo:** traslado al dominio transformado

In [167]:
A_hat <- hat_ten(A,M_l)
str(round(A_hat,4))

 num [1:4, 1:5, 1:3, 1:6, 1:2] 0.559 0.638 0.571 0.472 0.62 ...


**Función:** regreso al dominio original

In [168]:
inv_hat_ten <- function(A_hat,M_l) hat_ten(A_hat,lapply(M_l,solve))

**Ejemplo:** regreso al dominio original

In [169]:
A2 <- inv_hat_ten(A_hat,M_l)
str(A2)
paste("Error numérico:", sum(abs(A-A2))) #Error numérico

 num [1:4, 1:5, 1:3, 1:6, 1:2] 0.288 0.788 0.409 0.883 0.94 ...


[1] "Error numérico: 1.11642059529363e-12"

**Función:** cálculo del tensor transpuesto

In [170]:
t_ten <- function(A,M_l){
    dim_A <- dim(A)
    len_dim_A <- length(dim_A)
    A_hat <- hat_ten(A,M_l)
    A_hat_faces_T <- apply(A_hat, 3:len_dim_A, t, simplify = FALSE)
    A_hat_faces_T <- lapply(A_hat_faces_T, matrix, dim_A[2:1])
    A_hat_faces_T <- array(unlist(A_hat_faces_T),c(dim_A[2:1],dim_A[3:len_dim_A]))
    A_T_hat <- A_hat_faces_T
    inv_hat_ten(A_T_hat,M_l)
}

**Ejemplo:** cálculo del tensor transpuesto

In [171]:
A_T <- t_ten(A,M_l)
str(A_T)

 num [1:5, 1:4, 1:3, 1:6, 1:2] 0.288 0.94 0.551 0.678 0.246 ...


**Función:** cálculo del Producto $\star_M$

In [172]:
star_M <- function(A,B,M_l){
    inv_hat_ten(f_face_prod(hat_ten(A,M_l),hat_ten(B,M_l)),M_l)
}

**Ejemplo:** cálculo del Producto $\star_M$

In [173]:
str(star_M(A,B,M_l))

 num [1:4, 1:2, 1:3, 1:6, 1:2] -5.83 -15.37 -12.14 -3.43 10.07 ...


**Función:** cálculo de la norma de Frobenius

In [174]:
norm_frob <- function(A) norm(as.matrix(A),"F")

**Ejemplo:** cálculo de la norma de Frobenius

In [175]:
norm_frob(A)

[1] 15.42825

**Función:** obtención de la descomposición t-SVDM con base en el orden de los tubos singulares

In [176]:
tSVDM <- function(A,M_l){
    dim_A <- dim(A)
    len_dim_A <- length(dim_A)
    
    #Extraemos las caras de A_hat
    A_hat <- hat_ten(A,M_l)
    A_hat_faces <- apply(A_hat, 3:length(dim(A)), c, simplify = FALSE)
    A_hat_faces <- lapply(A_hat_faces, matrix, dim(A)[1:2])
    
    #Calculamos SVD para cada cara
    svd_l <- lapply(A_hat_faces,svd,nu=dim_A[1],nv=dim_A[2])
    
    #Formamos U_hat, S_hat y V_T_hat
    U_hat <- lapply(svd_l, function(l) l$u)
    U_hat <- array(unlist(U_hat), c(dim_A[1],dim_A[1],dim_A[-(1:2)]))
    
    fun_aux_S <- function(l){
        S_hat_j <- matrix(0, nrow = dim_A[1], ncol = dim_A[2])
        diag(S_hat_j) <- l$d
        S_hat_j
    }
    S_hat <- lapply(svd_l,fun_aux_S)
    S_hat <- array(unlist(S_hat), dim_A)
    
    V_T_hat <- lapply(svd_l, function(l) t(l$v))
    V_T_hat <- array(unlist(V_T_hat), c(dim_A[2],dim_A[2],dim_A[-(1:2)]))
    
    #Regresamos al dominio original
    U <- inv_hat_ten(U_hat,M_l)
    S <- inv_hat_ten(S_hat,M_l)
    V_T <- inv_hat_ten(V_T_hat,M_l)
    
    #Ordenamos los tensores U, S, V
    norm_S <- as.vector(diag(apply(S,1:2,norm_frob)))
    ord_S <- order(norm_S, decreasing = TRUE)
    rank_ten <- sum(norm_S!=0)
    
    dim_U <- dim(U)
    len_dim_U <- length(dim_U)
    
    U_to_parse <- paste0("U[,ord_S",paste0(rep(",",len_dim_U-2), collapse = ""),"]")
    U_ord <- eval(parse(text=U_to_parse))
    
    S_to_parse <- paste0("S[ord_S,ord_S",paste0(rep(",",len_dim_U-2), collapse = ""),"]")
    S_ord <- eval(parse(text=S_to_parse))
    
    V_T_to_parse <- paste0("V_T[ord_S,",paste0(rep(",",len_dim_U-2), collapse = ""),"]")
    V_T_ord <- eval(parse(text=V_T_to_parse))
    list(U=U_ord,S=S_ord,V_T=V_T_ord,rank=rank_ten)
}

**Ejemplo:** obtención de la descomposición t-SVDM con base en el orden de los tubos singulares

In [177]:
tsvdm <- tSVDM(A,M_l)
str(tsvdm)

paste("Error numérico:", sum(abs(A-star_M(star_M(tsvdm$U,tsvdm$S,M_l),tsvdm$V_T,M_l))))

List of 4
 $ U   : num [1:4, 1:4, 1:3, 1:6, 1:2] -0.1185 0.0336 -0.0355 -0.0156 8.9834 ...
 $ S   : num [1:4, 1:4, 1:3, 1:6, 1:2] 2.48 0 0 0 0 ...
 $ V_T : num [1:4, 1:5, 1:3, 1:6, 1:2] -0.0734 4.1659 -7.0652 -4.225 -0.0724 ...
 $ rank: int 4


[1] "Error numérico: 1.46905370897524e-11"